In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, RidgeCV,LogisticRegression,LinearRegression,Lasso
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer,QuantileTransformer, PowerTransformer, PolynomialFeatures,MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import xgboost as xgb
from catboost import CatBoostRegressor

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv', index_col='id')
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv', index_col='id')

In [ ]:
x = train.drop(columns=['target'])
y = train['target'].values
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3)

In [ ]:
model=CatBoostRegressor(iterations=10000,
                             learning_rate=0.001,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 23,
                             task_type="GPU",
                             od_type='Iter',
                             metric_period = 100,
                             od_wait=100)

In [ ]:
model.fit(x_train, y_train,
             eval_set=(x_val,y_val),
             use_best_model=True,
             verbose=True,plot=True)

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv', index_col='id')
sub['target'] = model.predict(test)
sub.to_csv('submission.csv')

In [ ]:
# import shap
# shap.initjs()
# explainer=shap.TreeExplainer(model)
# shap_values = explainer.shap_values(x)
# shap.force_plot(explainer.expected_value, shap_values[0,:], x.iloc[0,:])

In [ ]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.neighbors import KNeighborsRegressor
import lightgbm as lgb

In [ ]:
stack = StackingCVRegressor(regressors=(xgb.XGBRegressor(tree_method='gpu_hist', gpu_id=0), 
                            model),
                            meta_regressor=lgb.LGBMRegressor(device = 'gpu'), cv=20,
                            use_features_in_secondary=True,
                            store_train_meta_features=True,
                            shuffle=False,
                            random_state=1)

In [ ]:
stack.fit(x_train, y_train)

In [ ]:
test.columns

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv', index_col='id')
test.columns=['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13']
sub['target'] = stack.predict(test)
sub.to_csv('submissionboosters.csv')

## Underdevelopment******
THis is simple Catboost, I will be adding my hyperparameters and refining the data.